In [1]:
import os, glob
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import anndata as ad

In [2]:
dset_summary = pd.read_csv('../data/summary.csv', index_col = 0)
species_dict = dset_summary['species'].to_dict()
adata_fn = sorted(glob.glob('../data/processed/*/*.h5ad'))
adata_dict = {fn.split('/')[-1].split('_')[0] : fn for fn in adata_fn}
for key in adata_dict:
    adata_dict[key] = sc.read_h5ad(adata_dict[key])
    print(key, adata_dict[key])

GSE136689 AnnData object with n_obs × n_vars = 10097 × 17462
    obs: 'Stages', 'Clusters', 'Type', 'LineageAnnotations', 'celltype'
    var: 'n_cells', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    uns: 'Stages_colors', 'celltype_colors', 'diffmap_evals', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_diffmap', 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'
GSE162534 AnnData object with n_obs × n_vars = 8725 × 19013
    obs: 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'celltype'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    uns: 'celltype_colors', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'
GSE201257 Ann

In [3]:
adata_bulk_dict = adata_dict.copy()
for key in adata_dict:
    adata_key = adata_dict[key][~adata_dict[key].obs.celltype.isna()]
    adata_bulk_dict[key] = sc.get.aggregate(adata_key, 'celltype', 'mean')
    print(key, adata_bulk_dict[key])

GSE136689 AnnData object with n_obs × n_vars = 8 × 17462
    obs: 'celltype'
    var: 'n_cells', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    layers: 'mean'
GSE162534 AnnData object with n_obs × n_vars = 3 × 19013
    obs: 'celltype'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    layers: 'mean'
GSE201257 AnnData object with n_obs × n_vars = 13 × 21676
    obs: 'celltype'
    var: 'ERCC', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'n_cells', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    layers: 'mean'
GSE229103 AnnData object with n_obs × n_vars = 7 × 20088
    obs: 'celltype'
    var: 'n_cells', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'm', 'v', 'n_obs', 'res', 'lp', 'lpa', 'qv', 'highly_variable'
    layers: 'mean'
HCAEmbryonicHindlimb Ann

In [ ]:
signature_fn = sorted(glob.glob('../03_signatures/results/*/biomart/*.txt'))
signature_dict = {'_'.join([fn.split('/')[i] for i in [-3, -1]]).replace('.txt', '') : fn for fn in signature_fn}
for key in signature_dict:
    signature_dict[key] = pd.read_csv(signature_dict[key])
    print(key, signature_dict[key])

GSE136689_early    mmusculus hsapiens
0       Dab2     DAB2
1      Map1b    MAP1B
2       Irx1     IRX1
3      Rbm24    RBM24
4      Sox11    SOX11
..       ...      ...
65     Nop58    NOP58
66     Aldoa    ALDOA
67       Mmd      MMD
68   Slc16a3  SLC16A3
69    Mpped2   MPPED2

[70 rows x 2 columns]
GSE136689_late    mmusculus hsapiens
0        Id2      ID2
1     Homer2   HOMER2
2       Manf     MANF
3       Bex2     BEX1
4       Bex2     BEX2
5       Bex1     BEX1
6       Bex1     BEX2
7      Nr2f2    NR2F2
8       Wnt2     WNT2
9       Alx1     ALX1
10     Csrp2    CSRP2
11     Sfrp1    SFRP1
12      Krt8     KRT8
13     Krt18    KRT18
14    Phlda1   PHLDA1
15      Osr1     OSR1
16      Mycn     MYCN
17      Rbp1     RBP1
18     Gata6    GATA6
19     Sfrp5    SFRP5
20     Rras2    RRAS2
21     Pmp22    PMP22
22     Cox17    COX17
23    Popdc2   POPDC2
24       Ddt     DDTL
25       Ddt      DDT
26   Smarcd3  SMARCD3
27      Ly6e     LY6E
28      Nrp1     NRP1
29     Dusp9    DUSP9


In [5]:
df_ix = np.concatenate([adata_bulk_dict[key].obs.celltype for key in adata_bulk_dict])
assert np.unique(df_ix).size == df_ix.size
df_col = np.asarray(signature_dict.keys())
df = pd.DataFrame(0., index = df_ix, columns = df_col)
for key in adata_bulk_dict:
    adata = adata_bulk_dict[key]
    for col in df.columns:
        signature = signature_dict[col][species_dict[key]]
        df.loc[adata.obs.celltype, col] = sc.tl.score_genes(
            adata, signature, ctrl_as_ref = True, score_name = col,
            copy = True, layer = 'mean').obs[col]

       'mt-Cytb', 'Rps26'],
      dtype='object')
       'mt-Cytb'],
      dtype='object')


In [6]:
df_obs = np.concatenate([[key] * adata_bulk_dict[key].shape[0] for key in adata_bulk_dict])
df_obs = pd.DataFrame(df_obs, index = df.index, columns = ['source'])
df_var = np.flip(np.vstack(df.columns.str.split('_')), 1)
df_var = pd.DataFrame(df_var, index = df.columns, columns = ['stage', 'source'])
uns_dict = {
    'stage_colors' : list(sns.color_palette('Oranges', df_var.stage.nunique()).as_hex()),
    'source_colors' : list(sns.color_palette('cubehelix', df_var.source.nunique()).as_hex())
    }

In [7]:
adata = ad.AnnData(df, obs = df_obs, var = df_var, uns = uns_dict)
adata

AnnData object with n_obs × n_vars = 40 × 10
    obs: 'source'
    var: 'stage', 'source'
    uns: 'stage_colors', 'source_colors'

In [8]:
adata.write('results/adata_pseudobulk_signatures.h5ad')